## Forward Algorithmic Differentiation

Let us suppose that we have a known function $f: \mathbb{R}^{p_z} \to \mathbb{R}$. That is, for $z = (z_0, z_1, \dots, a_{p_z-1})^\top$, $f(z) = y \in \mathbb{R}$.

In this case the goal is to calculate the vector $\nabla f$ or $\frac{\partial f}{\partial z_i}, \forall i= 0, \dots, p_z-1$.

Suppose also that the function $f$ computationally is calculated in $p_b$ steps (i.e. $b_1, \dots, b_{p_b}=f$), thus we are going to calculate the derivatives with respect to these intermediate steps as well. Calling $\dot{b}[j,i]$ the derivative of the $j^{th}$ step/variable with respect to $z_i$, we have the following matrix

$$\dot{b} = \begin{array}{c|c|c|c|c|} 
&0 & 1 & \dots & p_z -1 \\ 
\hline \\ 
0 & 1 & 0 & \dots & 0 \\
\hline \\ 
1 & 0 & 1 & \dots & 0 \\
\hline \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
\hline \\ 
p_z-1 & 0 & 0 & \dots & 1 \\
\hline \\ 
p_z & \frac{\partial b_1}{\partial z_0} & \frac{\partial b_1}{\partial z_1} & \dots & \frac{\partial b_1}{\partial z_{p_z-1}} \\
\hline \\ 
p_z+1 & \frac{\partial b_2}{\partial z_0} & \frac{\partial b_2}{\partial z_1} & \dots & \frac{\partial b_2}{\partial z_{p_z-1}} \\
\hline \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
\hline \\ 
p_z+p_b-1 & \frac{\partial f}{\partial z_0} & \frac{\partial f}{\partial z_1} & \dots & \frac{\partial f}{\partial z_{p_z-1}} \\
\hline 
\end{array}
=
\begin{array}{|c|}
\hline
\dot{b}[0] \\
\hline
\dot{b}[1] \\
\hline
\dot{b}[2] \\
\hline
\dot{b}[3] \\
\hline
\dot{b}[4] \\
\hline
\dot{b}[5] \\
\hline
\dot{b}[6] \\
\hline
\dot{b}[7] \\
\hline
\end{array}
$$

Example:

$f(z_0, z_1, z_2, z_3) = \cos \left( z_0 + e^{z_1} \right) \left( \sin(z_2) + \cos(z_3) \right) +z_1^{3/2} + z_3$

Note that the gradient is easily calculated as:

$\nabla f = \begin{pmatrix} 
-\sin \left( z_0 + e^{z_1} \right) \left( \sin(z_2) + \cos(z_3) \right) \\ 
-\sin \left( z_0 + e^{z_1} \right) e^{z_1}\left( \sin(z_2) + \cos(z_3) \right) +\frac{3}{2} \sqrt{z_1}\\
\cos \left( z_0 + e^{z_1} \right) \cos(z_2)\\
-\cos \left( z_0 + e^{z_1} \right) \sin(z_3) + 1
\end{pmatrix} $

We divide the function $f$ into $4$ simple steps (i.e. $p_b =4$), that is 

In [ ]:
from math import *
 
def f(z):
  b4 = z[0] + exp(z[1])
  b5 = sin(z[2]) + cos(z[3])
  b6 = pow(z[1],1.5) + z[3]
  b7 = cos(b4)*b5 + b6
  return [b4,b5,b6,b7]

now set the vector of derivatives $\dot{b}$, this vector has dimension $(p_z+p_b) \times p_z$

In [ ]:
import numpy as np
z = np.array([1,0.2,0,0.5])
b = np.concatenate((z,f(z)), axis=0)
pz = 4
pb = 4
bdot = np.zeros((pz+pb,pz))
# derivatives of the variables
for i in range(pz):
  bdot[i,i] = 1
print(bdot)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


for the first line of derivatives of internal functions, (i.e. $b_4$) we have

$b_4(z) = z_0 + e^{z_1} $ thus its gradient $\nabla b_4 = (1,e^{z_1},0,0)^{\top}$

note that at this stage the row index is $i = p_z = 4$ thus we can use the chain rule and apply the gradient and the internal derivatives already calculated, that is

$\displaystyle \dot{b}[p_z,j] = \sum_{k=0}^{p_z-1} \frac{\partial b_{p_z}}{\partial b_k} \frac{\partial b_k}{\partial b_j} = \sum_{k=0}^{p_z-1} \frac{\partial b_{p_z}}{\partial b_k} \dot{b}[k,j]$

off course in this notation $b_k = a_k,$ if $k <p_a$.

And in this particular case

$\displaystyle \dot{b}[p_z,j] = \frac{\partial b_{p_z}}{\partial b_0} \dot{b}[0,j] + \frac{\partial b_{p_z}}{\partial b_1} \dot{b}[1,j] + \frac{\partial b_{p_z}}{\partial b_2} \dot{b}[2,j] + \frac{\partial b_{p_z}}{\partial b_3} \dot{b}[3,j] = 1 \dot{b}[0,j] + e^{b_1} \dot{b}[1,j] + 0 \dot{b}[2,j] + 0 \dot{b}[3,j]$


In [ ]:
i = pz
for j in range(i):
  bdot[i,j] = bdot[0,j] + exp(b[1])*bdot[1,j]
print(bdot)

[[1.         0.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         1.        ]
 [1.         1.22140276 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]


Applying the same idea, we have

$\displaystyle \dot{b}[5,j] = \sum_{k=0}^{4} \frac{\partial b_5}{\partial b_k} \frac{\partial b_k}{\partial z_j} $

That is (note the derivative with respect to $b_4$):

$\displaystyle \dot{b}[5,j] = \frac{\partial b_{5}}{\partial b_0} \dot{b}[0,j] + \frac{\partial b_{5}}{\partial b_1} \dot{b}[1,j] + \frac{\partial b_{5}}{\partial b_2} \dot{b}[2,j] + \frac{\partial b_{5}}{\partial b_3} \dot{b}[3,j] + \frac{\partial b_{5}}{\partial b_4} \dot{b}[4,j] $

And in this particular case

$\displaystyle \dot{b}[5,j] = \cos(b_2) \dot{b}[2,j] - \sin(b_3) \dot{b}[3,j]$

therefore we also have

$\displaystyle \dot{b}[6,j] = \frac{3}{2}\sqrt{b_1} \dot{b}[1,j] + \dot{b}[3,j]$

$\displaystyle \dot{b}[7,j] = -\sin(b_4) b_5 \dot{b}[4,j] + \cos(b_4) \dot{b}[5,j] + \dot{b}[6,j]$


In [ ]:
i += 1
for j in range(pz):
  bdot[i,j] = cos(b[2])*bdot[2,j] - sin(b[3])*bdot[3,j]
i += 1
for j in range(pz):
  bdot[i,j] = 1.5*sqrt(b[1])*bdot[1,j] + bdot[3,j]
i += 1
for j in range(pz):
  bdot[i,j] = -sin(b[4])*b[5]*bdot[4,j] + cos(b[4])*bdot[5,j] + bdot[6,j]
print(bdot)

[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]
 [ 1.          1.22140276  0.          0.        ]
 [ 0.          0.          1.         -0.47942554]
 [ 0.          0.67082039  0.          1.        ]
 [-0.69830705 -0.18209377 -0.60566906  1.29037322]]


The result is in the last line

In [ ]:
print(bdot[pz+pb-1])
print(cos(1+exp(0.2)))

[-0.69830705 -0.18209377 -0.60566906  1.29037322]
-0.6056690646099043
